In [1]:
import pandas as pd
import numpy as np
import re
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, LSTM, GRU, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')
# ---------------------------
# Step 1. Dataset Paths
# ---------------------------
train_path = r"adjectives_train.csv"
dev_path   = r"adjectives_dev.csv"
test_path  = r"adjectives_test.csv"

print("=== Loading Dataset ===")
train_df = pd.read_csv(train_path)
dev_df   = pd.read_csv(dev_path)
test_df  = pd.read_csv(test_path)

print(f"Train File: {train_path} -> {train_df.shape[0]} samples, {train_df.shape[1]} columns")
print(f"Dev File  : {dev_path} -> {dev_df.shape[0]} samples, {dev_df.shape[1]} columns")
print(f"Test File : {test_path} -> {test_df.shape[0]} samples, {test_df.shape[1]} columns\n")
# ---------------------------
# 3. Preprocessing
# ---------------------------
_stop = set(stopwords.words("english"))
_lem  = WordNetLemmatizer()

def preprocess_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"[^\w\s]", " ", text)
    tokens = word_tokenize(text)
    tokens = [_lem.lemmatize(w) for w in tokens if w not in _stop and w.strip()]
    return " ".join(tokens)

X_proc = X_text_raw.apply(preprocess_text).tolist()


# ---------------------------
# 4. Tokenization
# ---------------------------
MAX_NUM_WORDS = 30000
MAX_SEQ_LEN   = 200
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_proc)
X_seq = tokenizer.texts_to_sequences(X_proc)
X_pad = pad_sequences(X_seq, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')
VOCAB_SIZE = min(MAX_NUM_WORDS, len(tokenizer.word_index) + 1)


# ---------------------------
# 5. Deep feature extractors
# ---------------------------
FEATURE_DIM = 128
LR = 1e-3

def compile_model(inp, feat):
    out = Dense(NUM_CLASSES, activation='softmax')(feat)
    model = Model(inputs=inp, outputs=out)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(LR), metrics=['accuracy'])
    feat_extractor = Model(inputs=inp, outputs=feat)
    return model, feat_extractor

def build_cnn(max_len, vocab_size, emb_dim, feature_dim=FEATURE_DIM):
    inp = Input(shape=(max_len,))
    x = Embedding(vocab_size, emb_dim)(inp)
    x = Conv1D(128, 5, activation='relu', padding='same')(x)
    x = GlobalMaxPooling1D()(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    return compile_model(inp, feat)

def build_lstm(max_len, vocab_size, emb_dim, feature_dim=FEATURE_DIM):
    inp = Input(shape=(max_len,))
    x = Embedding(vocab_size, emb_dim)(inp)
    x = LSTM(128)(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    return compile_model(inp, feat)

def build_bilstm(max_len, vocab_size, emb_dim, feature_dim=FEATURE_DIM):
    inp = Input(shape=(max_len,))
    x = Embedding(vocab_size, emb_dim)(inp)
    x = Bidirectional(LSTM(128))(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    return compile_model(inp, feat)

def build_gru(max_len, vocab_size, emb_dim, feature_dim=FEATURE_DIM):
    inp = Input(shape=(max_len,))
    x = Embedding(vocab_size, emb_dim)(inp)
    x = GRU(128)(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    return compile_model(inp, feat)

def build_cnn_gru(max_len, vocab_size, emb_dim, feature_dim=FEATURE_DIM):
    inp = Input(shape=(max_len,))
    x = Embedding(vocab_size, emb_dim)(inp)
    x = Conv1D(128, 5, activation='relu', padding='same')(x)
    x = GRU(128)(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    return compile_model(inp, feat)

def build_cnn_lstm(max_len, vocab_size, emb_dim, feature_dim=FEATURE_DIM):
    inp = Input(shape=(max_len,))
    x = Embedding(vocab_size, emb_dim)(inp)
    x = Conv1D(128, 5, activation='relu', padding='same')(x)
    x = LSTM(128)(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    return compile_model(inp, feat)

BACKBONES = {
    "CNN": build_cnn,
    "LSTM": build_lstm,
    "BiLSTM": build_bilstm,
    "GRU": build_gru,
    "CNN-GRU": build_cnn_gru,
    "CNN-LSTM": build_cnn_lstm
}


# ---------------------------
# 6. Classical classifiers 
# ---------------------------
def build_classifiers(random_state=RANDOM_SEED):
    clfs = {
        "SVM-RBF": SVC(kernel="rbf", probability=True, random_state=random_state)
    }
    return clfs

# Parameter grid for hyperparameter tuning
param_grids = {
    "SVM-RBF": {
        "C": [0.1, 1, 10],
        "gamma": ["scale", "auto", 0.001, 0.0001]
    }
}
# ---------------------------
# ---------------------------
# 7. 10-Fold CV 
# ---------------------------
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from tensorflow.keras import backend as K

RANDOM_SEED = 42
NUM_CLASSES = len(train_df["Label"].unique())   # adjust column name if different

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_SEED)
fold_results = []

# Logging / output
os.makedirs("outputs", exist_ok=True)
per_fold_csv = "outputs/svm_rbf_10fold_results.csv"

for fold_idx, (tr_idx, val_idx) in enumerate(skf.split(X_pad, y), start=1):
    print(f"\n========== Fold {fold_idx}/10 ==========")
    X_tr, X_val = X_pad[tr_idx], X_pad[val_idx]
    y_tr, y_val = y[tr_idx], y[val_idx]
    y_tr_cat, y_val_cat = to_categorical(y_tr, num_classes=NUM_CLASSES), to_categorical(y_val, num_classes=NUM_CLASSES)

    # --- Deep feature extraction per backbone ---
    feats_tr_list, feats_val_list = [], []
    for name, builder in BACKBONES.items():
        print(f"Training backbone: {name}")
        model, feat_extractor = builder(MAX_SEQ_LEN, VOCAB_SIZE, EMBEDDING_DIM)
        es = EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True, verbose=0)
        model.fit(
            X_tr, y_tr_cat,
            validation_split=0.1,
            epochs=8,
            batch_size=64,
            callbacks=[es],
            verbose=0
        )
        feat_tr = feat_extractor.predict(X_tr, batch_size=128, verbose=0)
        feat_val = feat_extractor.predict(X_val, batch_size=128, verbose=0)
        feats_tr_list.append(feat_tr)
        feats_val_list.append(feat_val)

        # Free memory
        del model, feat_extractor
        K.clear_session()

    # Concatenate features from all backbones
    X_tr_feat = np.concatenate(feats_tr_list, axis=1)
    X_val_feat = np.concatenate(feats_val_list, axis=1)

    # Scale features (important for SVM)
    scaler = StandardScaler(with_mean=True, with_std=True)
    X_tr_feat = scaler.fit_transform(X_tr_feat)
    X_val_feat = scaler.transform(X_val_feat)

    # --- SVM-RBF classifier ---
    clf = SVC(kernel="rbf", C=1.0, gamma="scale", probability=True, random_state=RANDOM_SEED)
    clf.fit(X_tr_feat, y_tr)
    y_val_pred = clf.predict(X_val_feat)

    # --- Evaluate fold ---
    acc = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, average="weighted", zero_division=0)
    recall = recall_score(y_val, y_val_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_val, y_val_pred, average="weighted", zero_division=0)

    print(f"Fold {fold_idx} - Acc: {acc:.4f} | Prec: {precision:.4f} | Rec: {recall:.4f} | F1: {f1:.4f}")

    fold_results.append({
        "Fold": fold_idx,
        "Accuracy": acc,
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1
    })

    # Save incremental results
    pd.DataFrame(fold_results).to_csv(per_fold_csv, index=False)


# ---------------------------
# 8. Summary
# ---------------------------
        print(f"--- Fold {fold} Final ---")
        print(f"Accuracy: {acc_curve[end_epoch-1]:.2f}% | Precision: {prec_curve[end_epoch-1]:.2f}% "
              f"| Recall: {rec_curve[end_epoch-1]:.2f}% | F1: {f1_curve[end_epoch-1]:.2f}%")

    # Final CV Results
    print(f"\n>>> {model_name} Final CV Results ({FOLDS} folds)")
    print(f"Accuracy: {acc_final:.2f}")
    print(f"Precision: {prec_final:.2f}")
    print(f"Recall: {rec_final:.2f}")
    print(f"F1: {f1_final:.2f}")
    print("="*70)


=== Loading Dataset ===
Train File: adjectives_train.csv -> 6400 samples, 13 columns
Dev File  : adjectives_dev.csv -> 1600 samples, 13 columns
Test File : adjectives_test.csv -> 2000 samples, 13 columns


========== CNN-GRU Training (10-Fold CV) ==========

========== CNN-GRU | Fold 1/10 ==========
Epoch   1/100 - acc: 66.00% - prec: 69.88% - rec: 69.78% - f1: 70.12%
Epoch   2/100 - acc: 66.55% - prec: 70.41% - rec: 70.40% - f1: 70.49%
Epoch   3/100 - acc: 66.05% - prec: 70.37% - rec: 70.66% - f1: 70.50%
Epoch   4/100 - acc: 66.00% - prec: 71.10% - rec: 70.80% - f1: 70.78%
Epoch   5/100 - acc: 66.26% - prec: 70.87% - rec: 71.02% - f1: 70.98%
Epoch   6/100 - acc: 66.01% - prec: 71.55% - rec: 71.16% - f1: 71.10%
Epoch   7/100 - acc: 66.09% - prec: 71.47% - rec: 71.43% - f1: 71.71%
Epoch   8/100 - acc: 66.00% - prec: 72.20% - rec: 71.35% - f1: 72.06%
Epoch   9/100 - acc: 66.39% - prec: 71.79% - rec: 72.15% - f1: 72.19%
Epoch  10/100 - acc: 66.28% - prec: 72.48% - rec: 72.16% - f1: 72.23%